In [2]:
# Helpers
import numpy as np

# DB 
import psycopg2
from django.conf import settings

# Evaluation
from sklearn.metrics import roc_curve, auc

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

In [65]:
%%time
query = """select experiment_id, sp, sp_test, x, x_test, sp_hat, sk_hat, q_matrix_hat, mu
from EDM2020_2020_02_19 where method='fdtf' and dataset = 'run_0'"""
cursor.execute(query)
row = cursor.fetchone()

CPU times: user 19 s, sys: 9.79 s, total: 28.8 s
Wall time: 15min 28s


In [66]:
# %%time
update_queries = []
total = 0
while row:
    exp_id, sp, sp_test, X, X_test, sp_hat, sk_hat, q_matrix_hat, mu = row
    sp = np.asarray(sp)
    sp_test = np.asarray(sp_test)
    X = np.asarray(X)
    X_test = np.asarray(X_test)
    sp_hat = np.asarray(sp_hat)
    sk_hat = np.asarray(sk_hat)
    q_matrix_hat = np.asarray(q_matrix_hat)
    
    attempts_per_question = X.sum(axis=0).sum(axis=1)
    del_questions = [idx for idx,value in enumerate(attempts_per_question) if value < 3]
    X = np.delete(X, del_questions, axis=1)
    
    # Get train predicted values
    y = sp[np.where(X == 1)]
    y_pred = sp_hat[np.where(X == 1)]
    
    # Calculate AUC and threshold to binarize prediction
    fpr, tpr, thresholds = roc_curve(y, y_pred, pos_label=1)
    auc_score = auc(fpr, tpr)
    J_stats = tpr - fpr
    J_opt_thresholds = thresholds[np.argmax(J_stats)]
    y_pred_binary = np.where(np.asarray(y_pred) > J_opt_thresholds, 1, 0)
    
    # Calculate accuracy and RMSE
    acc = np.logical_not(np.logical_xor(y, y_pred_binary)).sum()/len(y)
    rmse = np.sqrt(np.power((y-y_pred), 2)).mean()
    
    update = {
        "train_auc": auc_score,
        "train_acc": acc,
        "train_rmse": rmse,
        "auc_threshold": J_opt_thresholds,
        "train_1":  np.where(y == 1)[0].shape[0],
        "train_0":  np.where(y == 0)[0].shape[0],
        "train_pred_1": np.where(y_pred_binary == 1)[0].shape[0],
        "train_pred_0": np.where(y_pred_binary == 0)[0].shape[0],
    }
    
    
    # Write PSQL query
    update_query = "UPDATE EDM2020_2020_02_19 SET "
    update_list = []
    query_values = []
    for key, value in update.items():
        update_list.append(key + "= %s")
        query_values.append(value)
    
    update_query += ", ".join(update_list) + "where experiment_id = %s"
    query_values.append(exp_id)
    query_values = tuple(query_values)
    query = cursor.mogrify(update_query, query_values)
    update_queries.append(query)
    total += 1
    try:
        row = cursor.fetchone()
    except psycopg2.ProgrammingError:
        row = False

In [70]:
for query in update_queries:
    cursor.execute(query)